In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
ROOT_DIR = r"/Users/jamesmoro/Documents/Python/ClimateData/data/interim/"
DF_NAMES = (
    "CH4_df",
    "CO2_df",
    "HFC_df",
    "NF3_df",
    "PFC_df",
    "SF6_df",
    "N20_df",
)

CH4_df = pd.read_pickle(ROOT_DIR + DF_NAMES[0] + ".pkl")
CO2_df = pd.read_pickle(ROOT_DIR + DF_NAMES[1] + ".pkl")
HFC_df = pd.read_pickle(ROOT_DIR + DF_NAMES[2] + ".pkl")
NF3_df = pd.read_pickle(ROOT_DIR + DF_NAMES[3] + ".pkl")
PFC_df = pd.read_pickle(ROOT_DIR + DF_NAMES[4] + ".pkl")
SF6_df = pd.read_pickle(ROOT_DIR + DF_NAMES[5] + ".pkl")
N20_df = pd.read_pickle(ROOT_DIR + DF_NAMES[6] + ".pkl")

In [3]:
list_of_dfs = [CH4_df,
CO2_df,
HFC_df,
NF3_df,
PFC_df,
SF6_df,
N20_df]

In [4]:
CO2_df.rename({"Country or Area":"Country", "Year":"Year", "Value":"CO2 Emissions"}, axis = 1, inplace = True)

In [5]:
for df, name in zip(list_of_dfs, DF_NAMES):
    df.rename({"Country or Area":"Country", "Year":"Year", "Value":f"{name[:3]} Emissions"}, axis = 1, inplace = True)

In [6]:
combined_df = pd.merge(CO2_df, NF3_df, how="outer")
combined_df = pd.merge(combined_df, CH4_df, how="outer")
combined_df = pd.merge(combined_df, HFC_df, how="outer")
combined_df = pd.merge(combined_df, PFC_df, how="outer")
combined_df = pd.merge(combined_df, SF6_df, how="outer")
combined_df = pd.merge(combined_df, N20_df, how="outer")


# Adding a "World" country

In [7]:
years = pd.Series(list(set(combined_df["Year"])), dtype = "int")

In [8]:
for year in years:
    _CO2 = combined_df[combined_df["Year"] == year]["CO2 Emissions"]
    _NF3 = combined_df[combined_df["Year"] == year]["NF3 Emissions"]
    _CH4 = combined_df[combined_df["Year"] == year]["CH4 Emissions"]
    _HFC = combined_df[combined_df["Year"] == year]["HFC Emissions"]
    _PFC = combined_df[combined_df["Year"] == year]["PFC Emissions"]
    _SF6 = combined_df[combined_df["Year"] == year]["SF6 Emissions"]
    _N20 = combined_df[combined_df["Year"] == year]["N20 Emissions"]
    for row, column in combined_df.iteritems():
        combined_dictionary = {"Country": "World", "Year": year, 
        "CO2 Emissions":np.nansum(_CO2),
          "NF3 Emissions":np.nansum(_NF3),
          "CH4 Emissions":np.nansum(_CH4),
          "HFC Emissions":np.nansum(_HFC),
          "PFC Emissions":np.nansum(_PFC),
          "SF6 Emissions":np.nansum(_SF6),
          "N20 Emissions":np.nansum(_N20),
          }
    combined_df = combined_df.append(combined_dictionary, ignore_index = True)

del _CO2,_NF3,_CH4,_HFC,_PFC,_SF6,_N20,combined_dictionary



In [9]:
combined_df["Total"] = combined_df.iloc[:,2:].sum(axis=1)

In [10]:
combined_df.head()

,Country,Year,CO2 Emissions,NF3 Emissions,CH4 Emissions,HFC Emissions,PFC Emissions,SF6 Emissions,N20 Emissions,Total
0,Australia,2018,415953.95,NaN,109532.21,11982.08,236.00,228.53,20114.44,558047.21
1,Australia,2017,415097.43,NaN,108170.70,11685.94,202.63,189.99,21265.59,556612.28
2,Australia,2016,411031.53,NaN,105873.93,11979.46,224.92,186.31,19566.39,548862.54
3,Australia,2015,401554.76,NaN,105368.80,11795.32,171.32,170.94,19557.63,538618.77
4,Australia,2014,394116.89,NaN,105070.91,10778.85,192.54,155.17,20096.66,530411.02


In [11]:
combined_df["CO2 Emissions"]

0         415953.95
1         415097.43
2         411031.53
3         401554.76
4         394116.89
           ...     
1271    17195134.41
1272    17070961.20
1273    16916947.71
1274    16943513.25
1275    16962390.04
Name: CO2 Emissions, Length: 1276, dtype: float64

In [103]:
combined_df.tail()

,Country,Year,CO2 Emissions,NF3 Emissions,CH4 Emissions,HFC Emissions,PFC Emissions,SF6 Emissions,N20 Emissions,Total
1271,World,2014,17195134.41,1780.35,2344635.43,474587.91,20613.78,22278.91,1174996.17,21234026.96
1272,World,2015,17070961.20,1281.42,2326847.30,471722.86,20374.57,21950.43,1170386.96,21083524.74
1273,World,2016,16916947.71,1351.54,2308602.32,471929.33,20525.42,22911.40,1160364.95,20902632.67
1274,World,2017,16943513.25,1179.40,2329363.28,481829.55,18984.39,23508.44,1168049.93,20966428.24
1275,World,2018,16962390.04,1048.65,2339998.69,478210.23,19371.03,23656.73,1174345.82,20999021.19


In [104]:
SAVE_DIR = r"/Users/jamesmoro/Documents/Python/ClimateData/data/interim"
file_path = Path(SAVE_DIR, "Combined_ghg_dataframe.pkl")

pd.to_pickle(combined_df, file_path)